# accre_jobs_2020

In [1]:
import pandas as pd

## Importing the Data

Check the `accre-jobs-2020--datasource-preprocessing.ipynb` file for the data pre-processing steps

In [2]:
data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
data.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


## Data Preparation

### Change headers to all lowercase

In [3]:
data.columns = data.columns.map(str.lower)
data.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


First, let's look at the quick summary

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   jobid      object
 1   account    object
 2   user       object
 3   reqmem     object
 4   usedmem    object
 5   reqtime    object
 6   usedtime   object
 7   nodes      int64 
 8   cpus       int64 
 9   partition  object
 10  exitcode   object
 11  state      object
 12  nodelist   object
dtypes: int64(2), object(11)
memory usage: 378.5+ MB


### Focusing on "production" partition only

We really only want to look at the "production" partition so rows with other partitions should be removed.

In [5]:
data_prod = data.loc[data["partition"] == "production"]
data_prod.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [6]:
data_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3311788 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   jobid      object
 1   account    object
 2   user       object
 3   reqmem     object
 4   usedmem    object
 5   reqtime    object
 6   usedtime   object
 7   nodes      int64 
 8   cpus       int64 
 9   partition  object
 10  exitcode   object
 11  state      object
 12  nodelist   object
dtypes: int64(2), object(11)
memory usage: 353.7+ MB


### How many are production nodes? Non-production nodes?

In [7]:
data["partition"].value_counts()

production              3311788
nogpfs                   327652
sam                       79151
pascal                    48004
turing                    39406
debug                      6738
maxwell                    3348
cgw-capra1                   83
cgw-dougherty1               72
cgw-cqs1                     34
cgw-vm-qa-flatearth1          4
cgw-cqs3                      4
cgw-tbi01                     2
devel                         2
cgw-rocksteady                2
Name: partition, dtype: int64

### Converting Job times to total seconds

Job time is in a format of either `d-hh:mm:ss` or `hh:mm:ss`, it needs to be converted to total seconds

In [8]:
def convert_time_to_seconds(time_str):
    """Convert the given time string d-hh:mm:ss or hh:mm:ss to total seconds."""
    
    # Account for possible errors
    try:
        
        # Initialize all time components to 0
        days = hours = minutes = seconds = 0

        # Split the time string at the dash mark
        time_dash_split = time_str.split("-") # => Either ["d", "hh:mm:ss"] or ["hh:mm:ss"]

        # Assign each time element to variables
        if len(time_dash_split) > 1: 
            # Remove the first element from the list and convert to integer and add to days
            days += int(time_dash_split.pop(0))

        # If here, len(time_dash_split) == 1
        # Split the time at the colon marks
        time_colon_split = time_dash_split[0].split(":") # => ["hh", "mm", "ss"]
        seconds += int(time_colon_split.pop())
        minutes += int(time_colon_split.pop())
        hours += int(time_colon_split.pop())

        # Now combine everything together
        return (((((days * 24) + hours) * 60) + minutes) * 60) + seconds
        
    except AttributeError as error:
        # Skip everything if the column is already in seconds (assuming one single digit)
        return time_str

Now, we can convert the time columns to seconds

In [9]:
new_reqtime = data["reqtime"].map(convert_time_to_seconds)
new_usedtime = data["usedtime"].map(convert_time_to_seconds)

data = data.assign(reqtime = new_reqtime)
data = data.assign(usedtime = new_usedtime)

data.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,1188000,1188028,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,1188000,1090256,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,1188000,1188020,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,1188000,1050632,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,1188000,1188003,1,24,production,0:0,COMPLETED,cn1440


### Remove `STATE == RUNNING`

Because these computers are still in their running state, their `usedmem` and `usedtime` are not accurate and could throw off our analysis

In [10]:
computers_state_running = data[data['state'] == 'RUNNING']
computers_state_running.shape

(3108, 13)

In [11]:
data = data[data['state'] != 'RUNNING']
data.shape

(3813182, 13)